# Pattern matching algorithms

## 1. Naive
#### Complexity: 
$O(m(n-m+1))$

In [1]:
def naive_string_matching(text, pattern):
    res = []
    for s in range(0, len(text) - len(pattern) + 1):
        if(pattern == text[s:s+len(pattern)]):
            res.append(s)
#     print("Pattern at index(es): ", res)
    return res

In [2]:
naive_string_matching("ababc", "abc")

[2]

In [3]:
naive_string_matching("sylwiasylwiasy", "ylw")

[1, 7]

## 2. Finite automata
#### Build transition table (TT): 
TT is table of size $(M+1)$ x $N$ where $M$ is length of pattern and $N$ is number of characters in pattern.

In given state $S$ and character $X$ next state (TT[$S$][$X$]) is length of longest prefix of pattern that is also suffix of pattern[$0 ... S-1$]$X$ (first $S$ letters of pattern concatenated with $X$)

#### Preprocessing complexity: 
$O(m^3|\Sigma|$), where $|\Sigma|$ is equals to alphabet size

#### Algorithm:
At every step consider next letter of given text and get next state from built TT (starting from state 0) and move to it.
Pattern is found if we reach last state ($M$).

#### Matching complexity:
$O(n)$

In [4]:
import re

In [5]:
def transition_table(pattern):
    result = []

    letters = set()

    for l in pattern:
        letters.add(l)

    for state in range(0, len(pattern) + 1):
        result.append({})
        for a in letters:
            # max possible next state 
            next_state = min(len(pattern), state + 1)
            
            while not (re.search(f"{pattern[:next_state]}$", pattern[:state] + a)):
                next_state -= 1
                
            result[state][a] = next_state   
    return result

In [6]:
delta = transition_table("kamk")
for i in delta:
    print(i)

{'m': 0, 'k': 1, 'a': 0}
{'m': 0, 'k': 1, 'a': 2}
{'m': 3, 'k': 1, 'a': 0}
{'m': 0, 'k': 4, 'a': 0}
{'m': 0, 'k': 1, 'a': 2}


In [7]:
def fa_string_matching(text, pattern, delta = False):
    res = []
    
    if not delta: delta = transition_table(pattern)
        
    state = 0
    for s in range(0, len(text)):
        # check if given letter s is present in pattern
        next_state = delta[state].get(text[s])
        if next_state is not None:
            state = next_state
            if(state == len(pattern)):
                res.append(s + 1 - state)
#     print("Pattern at index(es): ", res)
    return res

In [8]:
fa_string_matching("sylwiasylwiasy", "ylw")

[1, 7]

## 3. KMP
#### Build auxiliary LPS array:
LPS - Longest Proper Prefix that is also Suffix
LPS[$i$] stores length of longest proper prefix that is suffix of pattern[$0...i$]

#### Preprocessing complexity:
$O(m)$

#### Algorithm:
Increment indexes $q$ and $i$ while pattern[$q$] and text[$i$] keep matching/
If there is mismatch ...

#### Matching complexity:
$O(n)$

In [9]:
def prefix_function(pattern):
    lps = [None] * len(pattern)
    lps[0] = 0
    k = 0
    idx = 1
    for q in range(1, len(pattern)):
        while(k > 0 and pattern[k] != pattern[q]):
            k = lps[k-1]
        if(pattern[k] == pattern[q]):
            k = k + 1
        lps[idx] = k
        idx += 1
    return lps

In [10]:
def kmp_string_matching(text, pattern, lps = False):
    res = []
    if not lps: lps = prefix_function(pattern)
    q = 0
    for i in range(0, len(text)):
        while(q > 0 and pattern[q] != text[i]):
            q = lps[q-1]
        if(pattern[q] == text[i]):
            q = q + 1
        if(q == len(pattern)):
            res.append(i + 1 - q)
            q = lps[q-1]
#     print("Pattern at index(es): ", res)
    return res

In [11]:
prefix_function("ylw")

[0, 0, 0]

In [12]:
kmp_string_matching("sylwiasylwiasy", "ylw")

[1, 7]

### Algorithms comparison

In [13]:
with open("1997_714.txt", "r", errors = "ignore") as f:
    text = f.read()
pattern = "Art"

In [14]:
naive_string_matching(text, pattern)

[675,
 976,
 1665,
 2614,
 5703,
 6079,
 10923,
 11580,
 15144,
 16800,
 17127,
 24506,
 28085,
 28684,
 29378,
 31974,
 32644,
 33043,
 33922,
 35078,
 37790,
 39515,
 40362,
 43034,
 44860,
 51049,
 53106,
 56282,
 57672,
 59520,
 60705,
 62198,
 63645,
 64495,
 65449,
 66540,
 70136,
 70756,
 71376,
 72254,
 73969,
 74409,
 74943,
 75513,
 76671,
 78181,
 78868,
 79263,
 80089,
 80798,
 81059,
 82528,
 83572,
 88537,
 89650,
 90201,
 92748,
 93406]

In [15]:
fa_string_matching(text, pattern)

[675,
 976,
 1665,
 2614,
 5703,
 6079,
 10923,
 11580,
 15144,
 16800,
 17127,
 24506,
 28085,
 28684,
 29378,
 31974,
 32644,
 33043,
 33922,
 35078,
 37790,
 39515,
 40362,
 43034,
 44860,
 51049,
 53106,
 56282,
 57672,
 59520,
 60705,
 62198,
 63645,
 64495,
 65449,
 66540,
 70136,
 70756,
 71376,
 72254,
 73969,
 74409,
 74943,
 75513,
 76671,
 78181,
 78868,
 79263,
 80089,
 80798,
 81059,
 82528,
 83572,
 88537,
 89650,
 90201,
 92748,
 93406,
 243959,
 250724,
 252437]

In [16]:
kmp_string_matching(text, pattern)

[675,
 976,
 1665,
 2614,
 5703,
 6079,
 10923,
 11580,
 15144,
 16800,
 17127,
 24506,
 28085,
 28684,
 29378,
 31974,
 32644,
 33043,
 33922,
 35078,
 37790,
 39515,
 40362,
 43034,
 44860,
 51049,
 53106,
 56282,
 57672,
 59520,
 60705,
 62198,
 63645,
 64495,
 65449,
 66540,
 70136,
 70756,
 71376,
 72254,
 73969,
 74409,
 74943,
 75513,
 76671,
 78181,
 78868,
 79263,
 80089,
 80798,
 81059,
 82528,
 83572,
 88537,
 89650,
 90201,
 92748,
 93406]

## Execution time test


In [17]:
import time

In [18]:
def ex_test(text, pattern):
    start = time.time()
    naive_string_matching(text, pattern)
    end = time.time()
    print("Naive: ", end - start)
    
    start = time.time()
    fa_string_matching(text, pattern)
    end = time.time()
    print("FA: ", end - start)
    
    start = time.time()
    kmp_string_matching(text, pattern)
    end = time.time()
    print("KMP: ", end - start)

In [19]:
with open("1997_714.txt", "r", errors = "ignore") as f:
    text = f.read()

In [20]:
ex_test(text, "Art")

Naive:  0.08089756965637207
FA:  0.043701171875
KMP:  0.06016182899475098


### Execution time of 2. and 3. algorithm at least 5 times shorter than naive (without preprocessing)
Obs: In case where text has many matching letters with pattern followed by mismatching letter naive algorithm is really slow.
Another worst case is when all letters in text and in pattern are the same.

In [21]:
def ex_test_without_prep(text, pattern):
    start = time.time()
    naive_string_matching(text, pattern)
    end = time.time()
    naive_time = end - start
    print("Naive:", naive_time)
    
    delta = transition_table(pattern)
    start = time.time()
    fa_string_matching(text, pattern, delta)
    end = time.time()
    FA_time = end - start
    print("FA:   ", FA_time)
    
    lps = prefix_function(pattern)
    start = time.time()
    kmp_string_matching(text, pattern, lps)
    end = time.time()
    KMP_time = end - start
    print("KMP:  ", KMP_time)
    
    print("Naive / FA ratio:", round(naive_time/FA_time, 3))
    print("Naive / KMP ratio:", round(naive_time/KMP_time, 3))

In [22]:
# TODO!!!

text = "a"*1000
pattern = "a"*600 + "b"

In [23]:
ex_test_without_prep(text, pattern)

Naive: 0.0009984970092773438
FA:    0.0009949207305908203
KMP:   0.000997304916381836
Naive / FA ratio: 1.004
Naive / KMP ratio: 1.001


### Execution time of building transition table (2.) is at least 5 times longer than building LPS array (3.)

In [24]:
def ex_time_prep(text, pattern):
    start = time.time()
    delta = transition_table(pattern)
    end = time.time()
    FA_time = end - start
    print("FA prep:   ", FA_time)
    
    start = time.time()
    lps = prefix_function(pattern)
    end = time.time()
    KMP_time = end - start
    print("KMP prep:  ", KMP_time)
    
    print("FA prep / KMP prep ratio:", round(FA_time/KMP_time, 3))

In [25]:
text = "a"*100
pattern = "asbweqduvnakslgjpuogls"*20

In [28]:
ex_time_prep(text, pattern)

FA prep:    4.276195287704468
KMP prep:   0.0009982585906982422
FA prep / KMP prep ratio: 4283.655


In [29]:
# DOKONCZY: opis KMP i znalezc text + pattern 5 * szybszy + fa chyba sie zdupcyło XDXD